# 实验一
自定义深度卷积申请网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
#加载ImageDataGenerator模块
#加载定义CNN的layers、activate function等模块
#以下为将预测score转换为类别信息的to_categorical、图像预处理等函数
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

Using TensorFlow backend.


### 定义CNN网络结构
首先尝试：自定义深度CNN结构

然后尝试：定义Alex/VGG网络结构


In [2]:
#首先定义图像的长和宽，目的是告诉网络输入图像矩阵的大小，便于搭建网络结构
#以下为确认图像的通道分量
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
#在此定义网络结构    
model = Sequential()
#第一个 卷积层 32个kernel kernel大小3*3 输出的激活函数relu kernel利用 He-正态分布生成
model.add(Conv2D(32, (3, 3),input_shape = input_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2))) 

#在下述卷积层内32个kernel kernel大小3*3  在之后构建一个kernel size = 2 *2 的池化层 
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

#全链接前的一个操作，将feature map转成n*1的一维
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
#构建一个全联接，该全联接需要用特定的激活函数和适当的神经元个数 来实现我们的分类目标
model.add(Dense(1, activation='sigmoid'))    
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=0.0000001, decay=0.0)
model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])             
#model.add(BatchNormalization())


### 查看model架构



In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

### 定义ImageDataGenerator


In [4]:
#指定train和validation数据路
train_data_dir = r"E:\\input\\dogs-vs-cats\\train"
validation_data_dir = r"E:\\input\\dogs-vs-cats\\validation"
nb_train_samples = 10835
nb_validation_samples = 4000
#指定train和validation数据规模
epochs = 1
batch_size = 20

#声明ImageDataGenerator类型的train_datagen用于对training图像进行数据像素归一化和扩增
train_datagen = ImageDataGenerator(rescale = 1. / 255, shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True)

#声明ImageDataGenerator类型的test_datagen用于对validation图像进行像素归一化
test_datagen = ImageDataGenerator(rescale=1. /255)

#调用flow_from_directory生成train_generator
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                   target_size=(img_width, img_height),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')

#调用flow_from_directory生成validation_generator
validation_generator = train_datagen.flow_from_directory(validation_data_dir, 
                                                   target_size=(img_width, img_height),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')




Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [5]:
#调用并设置fit_generator函数训练模型
model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)



Epoch 1/1
541/541 [==============================] - 409s 757ms/step - loss: 0.6473 - accuracy: 0.6350 - val_loss: 0.6721 - val_accuracy: 0.6910


### 使用训练后模型预测图像





In [14]:
import cv2
#调用opencv的imread函数和resize函数读取测试样本并进行尺寸缩放，此处路径可更换
img = cv2.resize(cv2.imread(r'E:\input\dogs-vs-cats\test\229.jpg'), (img_width, img_height)).astype(np.float32)

#对图像像素矩阵进行tensor类型转换
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
#用训练好的模型对测试样本进行预测
score = model.predict(x)

print(score)

[[1.]]
